In [ ]:
from Config import BulkUploadConfig as Config

import sys
from flask_mysqldb import MySQL
import boto3
import os
import shutil
import numpy as np
import mysql.connector

from model.maskrcnn.predict_model import MaskRCNN_Predict as mask_rcnn_predict
from model.periwound.predict_model import PeriWound_Predict as peri_wound_predict
from model.wound_classification.predict_model import Wound_Classification as Wound_Classification
from model.Td_inpainting import main as ThreedModel
from model.wound_granulation.predict_model import Wound_Granulation as Wound_Granulation
from model.eschar.predict_model import Eschar_Predict as Eschar
from DBQueries import Queries as  DBQueries
from Utils import Utils

In [ ]:
mydb = mysql.connector.connect(
  host=Config.MYSQL_HOST,
  user=Config.MYSQL_USER,
  password=Config.MYSQL_PASSWORD,
  database=Config.MYSQL_DB
)

custom_session=boto3.session.Session(profile_name='default')
s3_client = custom_session.client('s3')

mask_rcnn_model_instance = mask_rcnn_predict(Config.WOUND_WEIGHTS_LOCATION,Config.WOUND_DIR)
peri_wound_model_instance = peri_wound_predict(Config.PERI_WOUND_WEIGHTS_LOCATION,Config.PERI_WOUND_DIR )
Wound_Granulation_instance =  Wound_Granulation(Config.WOUND_GRANULATION_WEIGHTS_LOCATION, Config.WOUND_GRANULATION_DIR )
eschar_model_instance = Eschar(Config.ESCHAR_WEIGHTS_LOCATION, Config.ESCHAR_DIR )
Wound_Classification_instance = Wound_Classification(Config.WOUND_CLASSIFICATION_WEIGHTS_LOCATION, Config.WOUND_CLASSIFICATION_DIR)


In [3]:
for wound_image in os.listdir(Config.IMAGE_FOLDER):
    try:
        wound_image = wound_image.lower()
        filename = wound_image
        upload_parameters = wound_image.split("_")
        doc_id = upload_parameters[0]
        patient_id = upload_parameters[1]
        wound_id = upload_parameters[2]
        upload_date = upload_parameters[3].replace(".jpg","")+' 00:00:00'

        
        upload_id = DBQueries.create_upload_id(mydb,doc_id,patient_id,upload_date,'Bulk Upload')
        DBQueries.insert_begin_end(mydb,wound_image,upload_id,'start')
        upload_ids_to_delete = DBQueries.get_upload_with_filename(mydb,filename)
        NoUploadsDeleted = DBQueries.delete_algo_uploads(mydb,upload_ids_to_delete)
        DBQueries.insert_into_filename(mydb,upload_id,filename)
    
        s3_client.upload_file(os.path.join(Config.IMAGE_FOLDER, filename), Config.S3_BUCKET, 'After_Cleaning/Raw/Image/'+filename)
        
        wound_predicted, TissueDistribution = mask_rcnn_model_instance.predict(Config.IMAGE_FOLDER,filename, Config.WOUND_LABELS_LOCATION, Config.WOUND_WEIGHTS_LOCATION, Config.WOUND_DIR, Config.WOUND_OUTPUT_FOLDER ,s3_client,Config.S3_BUCKET,"After Cleaning")
        DBQueries.insert_into_wound(mydb,wound_predicted,upload_id,wound_id,"After Cleaning")
        #DBQueries.insert_into_characterstics_color_model(mydb,TissueDistribution,upload_id)
       
       
        peri_wound_predicted = peri_wound_model_instance.predict(Config.IMAGE_FOLDER,filename,Config.PERI_WOUND_LABELS_LOCATION,Config.PERI_WOUND_WEIGHTS_LOCATION ,Config.PERI_WOUND_DIR ,Config.PERI_WOUND_OUTPUT_FOLDER,Config.WOUND_OUTPUT_FOLDER,s3_client,Config.S3_BUCKET,"After Cleaning")
        DBQueries.insert_into_periwound(mydb,peri_wound_predicted,upload_id,wound_id,"After Cleaning")

        eschar_predicted = eschar_model_instance.predict(Config.IMAGE_FOLDER,filename,Config.ESCHAR_LABELS_LOCATION, Config.ESCHAR_WEIGHTS_LOCATION, Config.WOUND_OUTPUT_FOLDER, Config.STATIC_IMAGES)
        DBQueries.insert_into_characterstics(mydb,eschar_predicted,upload_id,wound_id,"After Cleaning")

        wound_classification_predicted = Wound_Classification_instance.predict(Config.IMAGE_FOLDER,filename,Config.WOUND_CLASSIFICATION_LABELS_LOCATION, Config.WOUND_CLASSIFICATION_WEIGHTS_LOCATION, Config.WOUND_CLASSIFICATION_DIR, Config.WOUND_CLASSIFICATION_OUTPUT_FOLDER ,s3_client,Config.S3_BUCKET ,Config.WOUND_OUTPUT_FOLDER, Config.PERI_WOUND_OUTPUT_FOLDER ,Config.STATIC_IMAGES)
        DBQueries.insert_into_characterstics(mydb,wound_classification_predicted,upload_id,wound_id,"After Cleaning")

        wound_granulation_predicted = Wound_Granulation_instance.predict(Config.IMAGE_FOLDER,filename,Config.WOUND_GRANULATION_LABELS_LOCATION, Config.WOUND_GRANULATION_WEIGHTS_LOCATION, Config.WOUND_GRANULATION_DIR, Config.WOUND_CLASSIFICATION_OUTPUT_FOLDER ,s3_client,Config.S3_BUCKET ,Config.WOUND_OUTPUT_FOLDER,Config.PERI_WOUND_OUTPUT_FOLDER,Config.STATIC_IMAGES,"After Cleaning")
        DBQueries.insert_into_characterstics(mydb,wound_granulation_predicted,upload_id,wound_id,"After Cleaning")
        print(NoUploadsDeleted)
        print("****************")

        try:
            f = open(os.path.join(Config.DEPTH_OUTPUT,  filename.replace(".jpg",".npy")), "r");
            f.close();
        except (FileNotFoundError):
            shutil.copy(Config.IMAGE_FOLDER +"/" +filename,Config.THREEDMODEL_BASE+'/image/'+filename)
            ThreedModel.run(Config.THREEDMODEL_BASE, Config.MESH_OUTPUT, Config.DEPTH_OUTPUT)  
            s3_client.upload_file(os.path.join(Config.DEPTH_OUTPUT, filename.replace(".jpg",".npy")), Config.S3_BUCKET, 'After_Cleaning/Algo/Depth/'+filename.replace(".jpg",".npy"))
            s3_client.upload_file(os.path.join(Config.MESH_OUTPUT, filename.replace(".jpg",".ply")), Config.S3_BUCKET, 'After_Cleaning/Algo/PLY/'+filename.replace(".jpg",".ply") )

        DBQueries.UpdateDepth(mydb,upload_id,os.path.join(Config.DEPTH_OUTPUT,  filename.replace(".jpg",".npy")))
        Utils.clear_all_files(filename,Config)

        DBQueries.insert_begin_end(mydb,wound_image,upload_id,'end')
    
    except Exception as e:
        print(e)
        shutil.copy(Config.IMAGE_FOLDER +"/" +filename,Config.ERROR_IMAGES+'/'+filename)
        os.unlink(Config.IMAGE_FOLDER +"/" +filename)
        DBQueries.delete_algo_uploads(mydb,upload_id)
       
    


doc1_p22_13_2020_09_01.jpg
Processing 1 images
image                    shape: (1008, 756, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  146.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
                         Cnt
Tag                         
Dark Gray                  5
Eschar                   295
Gray                    7819
Healthy Granulation    23844
Slough                 12560
UnHealthy Granulation   3190
Processing 1 images
image                    shape: (1008, 756, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  146.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (

                         Cnt
Tag                         
Gray                    4069
Healthy Granulation     5279
Slough                 30770
UnHealthy Granulation    485
Processing 1 images
image                    shape: (1008, 756, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  149.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (1008, 756, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  149.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
0
****************
An 

In [ ]:
print("Done")

In [ ]:
DBQueries.insert_begin_end(mydb,wound_image,upload_id,'end')

In [ ]:
s3_client.download_file(Config.S3_BUCKET, 'After_Cleaning/Algo/Depth/'+filename.replace(".jpg",".npy"),os.path.join(Config.DEPTH_OUTPUT, filename.replace(".jpg",".npy")))

# Step By Step run of Algorithm

In [ ]:
wound_image = os.listdir(Config.IMAGE_FOLDER)[0].lower()
filename = wound_image
upload_parameters = wound_image.split("_")
doc_id = upload_parameters[0]
patient_id = upload_parameters[1]
wound_id = upload_parameters[2]
upload_date = upload_parameters[3].replace(".jpg","")+' 00:00:00'

    
upload_id = DBQueries.create_upload_id(mydb,doc_id,patient_id,upload_date,'Bulk Upload')
DBQueries.insert_begin_end(mydb,wound_image,upload_id,'start')
upload_ids_to_delete = DBQueries.get_upload_with_filename(mydb,filename)



In [ ]:
DBQueries.insert_into_filename(mydb,upload_id,filename)

In [ ]:
DBQueries.delete_algo_uploads(mydb,upload_ids_to_delete)
DBQueries.insert_into_filename(mydb,upload_id,filename)

In [ ]:
RunAlgos(filename,Config,ImageFolder,"After Cleaning")

In [ ]:
def RunAlgos(filename,Config,ImageFolder,image_type):
    wound_predicted = mask_rcnn_model_instance.predict(Config.IMAGE_FOLDER,filename, Config.WOUND_LABELS_LOCATION, Config.WOUND_WEIGHTS_LOCATION, Config.WOUND_DIR, Config.WOUND_OUTPUT_FOLDER ,s3_client,Config.S3_BUCKET,"AC")
    DBQueries.insert_into_wound(mydb,wound_predicted,upload_id,wound_id,"After Cleaning")

    peri_wound_predicted = peri_wound_model_instance.predict(Config.IMAGE_FOLDER,filename,Config.PERI_WOUND_LABELS_LOCATION,Config.PERI_WOUND_WEIGHTS_LOCATION ,Config.PERI_WOUND_DIR ,Config.PERI_WOUND_OUTPUT_FOLDER,Config.WOUND_OUTPUT_FOLDER,s3_client,Config.S3_BUCKET,"AC")
    DBQueries.insert_into_periwound(mydb,peri_wound_predicted,upload_id,wound_id,"After Cleaning")

    eschar_predicted = eschar_model_instance.predict(Config.IMAGE_FOLDER,filename,Config.ESCHAR_LABELS_LOCATION, Config.ESCHAR_WEIGHTS_LOCATION, Config.WOUND_OUTPUT_FOLDER, Config.STATIC_IMAGES)
    DBQueries.insert_into_characterstics(mydb,eschar_predicted,upload_id,wound_id,"After Cleaning")

    wound_classification_predicted = Wound_Classification_instance.predict(Config.IMAGE_FOLDER,filename,Config.WOUND_CLASSIFICATION_LABELS_LOCATION, Config.WOUND_CLASSIFICATION_WEIGHTS_LOCATION, Config.WOUND_CLASSIFICATION_DIR, Config.WOUND_CLASSIFICATION_OUTPUT_FOLDER ,s3_client,Config.S3_BUCKET ,Config.WOUND_OUTPUT_FOLDER, Config.PERI_WOUND_OUTPUT_FOLDER ,Config.STATIC_IMAGES)
    DBQueries.insert_into_characterstics(mydb,wound_classification_predicted,upload_id,wound_id,"After Cleaning")

    wound_granulation_predicted = Wound_Granulation_instance.predict(Config.IMAGE_FOLDER,filename,Config.WOUND_GRANULATION_LABELS_LOCATION, Config.WOUND_GRANULATION_WEIGHTS_LOCATION, Config.WOUND_GRANULATION_DIR, Config.WOUND_CLASSIFICATION_OUTPUT_FOLDER ,s3_client,Config.S3_BUCKET ,Config.WOUND_OUTPUT_FOLDER,Config.PERI_WOUND_OUTPUT_FOLDER,Config.STATIC_IMAGES,"AC")
    DBQueries.insert_into_characterstics(mydb,wound_granulation_predicted,upload_id,wound_id,"After Cleaning")

    shutil.copy(Config.IMAGE_FOLDER +"/" +filename,Config.THREEDMODEL_BASE+'/image/'+filename)
    ThreedModel.run(Config.THREEDMODEL_BASE, Config.MESH_OUTPUT, Config.DEPTH_OUTPUT)  

    s3_client.upload_file(os.path.join(Config.DEPTH_OUTPUT, filename.replace(".jpg",".npy")), Config.S3_BUCKET, 'After_Cleaning/Algo/Depth/'+filename.replace(".jpg",".npy"))
    s3_client.upload_file(os.path.join(Config.MESH_OUTPUT, filename.replace(".jpg",".ply")), Config.S3_BUCKET, 'After_Cleaning/Algo/PLY/'+filename.replace(".jpg",".ply") )

    DBQueries.UpdateDepth(mydb,os.path.join(Config.DEPTH_OUTPUT, filename.replace(".jpg",".npy")))

    Utils.clear_all_files(filename,Config)